In [ ]:
! pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/ece1786-project

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/ece1786-project


In [ ]:
import transformers
from datasets import Dataset
from transformers import AutoTokenizer
import pandas as pd

In [ ]:
model_checkpoint = "gpt2" #"distilgpt2"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2", use_fast=True)

In [ ]:
df = pd.read_pickle("cake.pkl")  
recipes = []
for i in range(len(df.index)):
    recipe = "RECIPE NAME\n" + df.loc[i, "Name"] + " \n\nRECIPE INGREDIENTS\n"
    skip = False
    for ingredient in df.loc[i, "Ingredients"]:
        '''
        # remove asterisks
        ingredient.replace('*', '')
        if "750" in ingredient:
            #skip batch drink recipes
            skip = True
        '''
        recipe += ingredient + "\n"
    if skip:
        continue
    recipe += "\nRECIPE INSTRUCTIONS\n" 
    for instruction in  df.loc[i, "Instructions"]:
        if instruction.startswith("*"):
            continue
        recipe += instruction + "\n"
    recipes.append(recipe)
tokenizer.pad_token = tokenizer.eos_token
data = tokenizer(recipes, padding='longest', truncation=True)
data["labels"] = data["input_ids"].copy()

dataset = Dataset.from_dict(data)
print(len(dataset))
print(dataset[5])
print(dataset)

386
{'input_ids': [38827, 4061, 36, 36751, 198, 51, 10757, 30799, 220, 198, 198, 38827, 4061, 36, 3268, 10761, 1961, 40, 15365, 198, 17, 14180, 477, 12, 29983, 10601, 198, 16, 22326, 16871, 11913, 198, 19, 220, 9653, 198, 17, 30064, 6145, 34569, 198, 16, 6508, 7545, 198, 17, 8059, 7545, 11311, 18550, 2318, 11, 20720, 198, 16, 1587, 120, 14180, 26636, 9215, 198, 16, 22326, 16858, 7925, 198, 17, 14180, 2330, 7543, 198, 198, 38827, 4061, 36, 3268, 46126, 11053, 198, 6719, 25080, 14361, 284, 13803, 7370, 376, 13, 11955, 589, 290, 10601, 257, 838, 87, 1314, 11111, 3425, 13, 311, 2135, 1978, 262, 10601, 290, 16871, 11913, 26, 900, 7263, 13, 198, 818, 257, 1588, 9396, 11, 12082, 9653, 11, 7543, 290, 16858, 13, 12568, 319, 1029, 2866, 286, 281, 5186, 33938, 1566, 1657, 290, 18873, 12, 25717, 13, 12568, 287, 262, 10601, 11710, 13, 7631, 808, 1015, 7545, 290, 6145, 34569, 546, 362, 2431, 11, 393, 1566, 340, 6140, 284, 14310, 13, 33689, 656, 4984, 13, 198, 47, 454, 4984, 656, 838, 87, 1314, 11111

In [ ]:
print(tokenizer.decode(dataset[15]["input_ids"]))
print(len(dataset[5]["input_ids"]))

RECIPE NAME
Moist Yellow Cake 

RECIPE INGREDIENTS
3 ¾ cups all-purpose flour
2 ¼ teaspoons baking powder
2 ½ teaspoons baking soda
1 cup butter
2 ½ cups buttermilk
3  eggs
1 ½ teaspoons vanilla extract
2 ½ cups white sugar

RECIPE INSTRUCTIONS
Preheat oven to 350 degrees F. Grease and flour two 9x13 inch pans. Sift together the flour, baking powder and baking soda. Set aside.
In a large bowl, cream together the butter and sugar until light and fluffy. Beat in the eggs one at a time, then stir in the vanilla. Beat in the flour mixture alternately with the buttermilk, mixing just until incorporated. Pour batter into prepared pans.
Bake in the preheated oven for 35 to 40 minutes, or until a toothpick inserted into the center of the cake comes out clean. Allow to cool.
<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|

In [ ]:
from transformers import AutoModelForCausalLM
from transformers import Trainer, TrainingArguments
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-cake",
    save_strategy="steps",
    save_steps=750,
    save_total_limit=1,
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=50,
    per_device_train_batch_size=2
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 386
  Num Epochs = 50
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 9650
  Number of trainable parameters = 124439808


Step,Training Loss
500,0.586200
1000,0.457300
1500,0.425800
2000,0.393700
2500,0.376200
3000,0.349200
3500,0.339300
4000,0.319100
4500,0.304900
5000,0.295600


Saving model checkpoint to gpt2-finetuned-cake/checkpoint-750
Configuration saved in gpt2-finetuned-cake/checkpoint-750/config.json
Model weights saved in gpt2-finetuned-cake/checkpoint-750/pytorch_model.bin
Deleting older checkpoint [gpt2-finetuned-cake/checkpoint-4500] due to args.save_total_limit
Deleting older checkpoint [gpt2-finetuned-cake/checkpoint-5250] due to args.save_total_limit
Saving model checkpoint to gpt2-finetuned-cake/checkpoint-1500
Configuration saved in gpt2-finetuned-cake/checkpoint-1500/config.json
Model weights saved in gpt2-finetuned-cake/checkpoint-1500/pytorch_model.bin
Deleting older checkpoint [gpt2-finetuned-cake/checkpoint-750] due to args.save_total_limit
Saving model checkpoint to gpt2-finetuned-cake/checkpoint-2250
Configuration saved in gpt2-finetuned-cake/checkpoint-2250/config.json
Model weights saved in gpt2-finetuned-cake/checkpoint-2250/pytorch_model.bin
Deleting older checkpoint [gpt2-finetuned-cake/checkpoint-1500] due to args.save_total_limit

Step,Training Loss
500,0.586200
1000,0.457300
1500,0.425800
2000,0.393700
2500,0.376200
3000,0.349200
3500,0.339300
4000,0.319100
4500,0.304900
5000,0.295600




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=9650, training_loss=0.32287442854649046, metrics={'train_runtime': 7003.6444, 'train_samples_per_second': 2.756, 'train_steps_per_second': 1.378, 'total_flos': 1.00858724352e+16, 'train_loss': 0.32287442854649046, 'epoch': 50.0})

In [ ]:
prompt = "RECIPE NAME\n"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cpu")

outputs = model.generate(input_ids, do_sample=True, max_length=600, temperature = 0.5, top_p = 1)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.8/dist-packages/transformers/generation_utils.py:1442: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [ ]:
from transformers import AutoModelForCausalLM
model_checkpoint = "gpt2-finetuned-cake/checkpoint-9000"
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained("gpt2", use_fast=True)

loading configuration file gpt2-finetuned-cake/checkpoint-9000/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2-finetuned-cake/checkpoint-9000",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "use_c

In [ ]:
prompt = "RECIPE NAME\n"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cpu")

outputs = model.generate(input_ids, do_sample=True, max_length=600,temperature = 0.5, top_p = 1)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


RECIPE NAME
Pumpkin Crunch Cake 

RECIPE INGREDIENTS
1 cup chopped pecans
1 can crushed pineapple, drained
2  eggs
2 cups evaporated milk
1 cup margarine
1 teaspoon pumpkin pie spice
1 can pumpkin puree
1 teaspoon salt
1 ½ cups white sugar

RECIPE INSTRUCTIONS
Preheat the oven to 350 degrees F. Grease and flour a 9x13-inch baking dish.
Make crust: Combine pumpkin puree, evaporated milk, sugar, eggs, pumpkin pie spice, and salt in a bowl; mix well. Press mixture into the bottom of the prepared dish.
Make topping: Combine evaporated milk, melted butter, and pumpkin puree in a saucepan over medium heat; cook and stir until thickened. Pour topping over crust.
Bake in the preheated oven until a toothpick inserted into the center comes out clean, about 40 minutes.



In [ ]:
prompt1 = "RECIPE NAME\nMagnum Opus"
input_ids1 = tokenizer(prompt1, return_tensors="pt").input_ids.to("cpu")

outputs1 = model.generate(input_ids1, do_sample=True, max_length=600, temperature = 0.6, top_p = 1, return_dict_in_generate=True, output_scores=True)
print(tokenizer.batch_decode(outputs1.sequences, skip_special_tokens=True)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


RECIPE NAME
Magnum Opus Cake 

RECIPE INGREDIENTS
1 cup all-purpose flour
3 teaspoons baking powder
1 ½ teaspoons baking soda
1 cup butter, softened
2  eggs
2 ½ cups grated orange zest
½ teaspoon ground cinnamon
½ teaspoon ground ginger
½ teaspoon ground nutmeg
2 ¾ cups heavy whipping cream
1 ½ cups packed brown sugar
¾ cup packed dark brown sugar
½ teaspoon salt
1 teaspoon vanilla extract
½ cup vegetable oil
2 cups white sugar

RECIPE INSTRUCTIONS
Preheat the oven to 350 degrees F. Grease and flour a 9x13-inch baking dish.
Beat white sugar and brown sugar together in a bowl using an electric mixer until light and fluffy. Add eggs, 1 at a time, beating well after each addition. Add oil and vanilla extract; mix until just combined. Add flour, baking powder, baking soda, cinnamon, ginger, nutmeg, and salt; mix until just combined. Pour batter into the prepared pan.
Bake in the preheated oven until a toothpick inserted into the center comes out clean, about 1 hour. Cool cake completely on